# Directory of Consumption-Saving Models in HARK

HARK has *many* consumption-saving models in several files in `HARK.ConsumptionSaving`. This notebook provides a directory of all of them, ordered (roughly) from the most basic to the most niche. We provide only brief textual descriptions of the models here. The full mathematical representation for each model can be found in its respective file and/or at the linked reference notebooks.

## `ConsIndShockModel`: Models With(out) Idiosyncratic Income Risk and One Risk-Free Asset

The model(s) that started it all! `ConsIndShockModel` has "canonical" consumption-saving models with the most fundamental / core assumptions. Most HARK models descend from the models here, in one way or another.

Critically, these models are *permanent income normalized*: their state space and consumption function are expressed in terms of dollar values divided by permanent income. Actual *levels* can be recovered by multiplying by permanent income. Mathematically speaking, these models are homothetic with respect to permanent income.

### `PerfForesightConsumerType`: Consumption-saving with no financial risk at all

This class represents consumers who can save in a single risk-free asset and who face mortality risk, but who face no income risk. Like all HARK models, these consumers have CRRA preferences over consumption and discount utility geometrically.

The default parameters do not include a borrowing constraint, but the model *can* handle this feature and solve for the consumption function exactly (up to machine precision).

### `IndShockConsumerType`: Consumption-saving with permanent-transitory income risk

This is HARK's core "workhorse" model, and the one for which the most theory work has been done. Agents in this model face iid income shocks each period, with two components: a fully *permanent* shock that affects their income in this period and all future periods, and a fully *transitory* component that only affects income this period.

Chris Carroll has spent a lot of time pinning down the theoretical properties of this model, and HARK's solver incorporates a lot of that mathematical material. Particularly, the consumption function will always obey theoretical constraints with respect to the perfect foresight solution, as well as boundary behavior as market resources go to their minimum level.

### `KinkedRconsumerType`: Consumption-saving with a higher interest rate for borrowing

The agents in this model face the exact same problem as those in `IndShockConsumerType` *except* that there is a higher interest rate for *borrow* than for *saving*. This creates two kinks in the consumption function, with a $c=m$ segment in between.

Notably, the first order condition does not hold with equality on the $c=m$ segment. Instead, the marginal value of assets is too low to *save* any money, but it is also too high to *borrow* money.

## `GenIncProcessModel`: Non-Permanent-Income-Normalized Models

In this module, we lift the assumption of a permanent-transitory income shock structure and allow for a *generalized* income process. The key difference is that the permanent (or persistent) component of income now appears explicitly as a state variable in the policy function (and other components of the solution).

### `GenIncProcessConsumerType`: Generalized income processes

This is the base `AgentType` subclass for this module; the other subclasses only extend it by setting different default constructors.

The permanent-income-normalizable income process is characterized by a sequence of expected permanent income growth factors and joint distributions of permanent and transitory income shocks. In contrast, the "general" income process is characterized by a sequence of "expected future income functions". These map from current `pLvl` values to succeeding period `pLvl' values, as a general function.

In this model, `pLvl` isn't "permanent" income *per se*, but instead a univariate measure of some semi-permanent or persistent component. The key restriction is that the *continuation-relevant* information has only one dimension after income shocks have been realized.

For ease of compatibility, the two components of the income shock distribution are still labeled `TransShk` and `PermShk`, but the latter affects `pLvl` multiplicatively, no matter the interpretation of `pLvl` for your setting. The transitory shock is, of course, still fully transitory.

### `IndShockExplicitPermIncConsumerType`: Workhorse model with explicit permanent income

This `AgentType` is meant to reproduce the same model as `IndShockConsumerType`, but maintains permanent income as an explicit state variable. This class mainly exists as a benchmark comparison.

Note that the solution for this class will be *less accurate* than the analogous one for `IndShockConsumerType` because the grid of permanent income values is necessarily bounded, but actual permanent income can grow (or shrink!) without bound. Thus there is extrapolation error near the top and bottom of the permanent income grid, which propagates throughout the solution.

### `PersistentShockConsumerType`: Consumption-saving model with AR(1) income process

This subclass is simply `GenIncProcessConsumerType` with an AR(1) process (in logs) used for the persistent component of income. This model is good for working with highly persistent (but not permanent) income shocks. Because the model is not normalized, it can also be adapted to incorporate a realistic income tax structure and/or Social Security system.

We think that reasonable values of the (annual) correlation or persistence coefficient are between 0.97 and 0.995, but we won't judge any values that you use.

## `ConsMarkovModel`: Models with a discrete state that evolves exogenously

This module has only one `AgentType` subclass, `MarkovConsumerType`, but it is arguably one of the most flexible models in HARK. It extends `IndShockConsumerType` by incorporating a discrete state that changes each period based on some Markov matrix.

Most elements of the consumer's problem can vary by discrete state, including the interest factor, expected permanent income growth, survival probability, and the distribution of income shocks.

Depending on how you specify or parameterize the model, `MarkovConsumerType` can represent many different kinds of models.  It can be used to represent a model with persistent unemployment, or one whose income shock distribution has a more complicated structure than permanent-transitory. It can be used to solve models with persistent variation in returns to saving, or as a "cheap" measure of health with variable survival probabilities. There are many more strange and unusual specifications, as well.

## `TractableBufferStockModel`: Illustrative model for learning consumption theory

Famously, consumption-saving models with income risk (and risk-averse agents) have no closed form solution form for their optimal consumption function prior to the terminal period. Some theoretical results to characterize the consumption function and how it changes with model parameters are fairly easy to derive and can be demonstrated for students, but others are significantly trickier.

Chris Carroll developed the "tractable buffer stock" model as a teaching tool for learning the theory of consumption-saving models. It has a very specific (and extreme) form of income uncertainty: the consumer faces a constant risk each period of becoming *permanently* unemployed and *never* receiving labor income again. Otherwise, their labor income grows by a constant factor.

The model for `TractableConsumerType`s doesn't *quite* have a closed form solution, but *more* of its characteristics and comparative statics can be derived on paper. Moreover, the solution method for *efficiently* constructing the consumption function is quite different to other HARK models. Broadly, the model is (more or less) only for teaching purposes, not for "serious" applied work.

## `ConsRepAgentModel`: What's this doing here?

HARK is nominally for heterogeneous agents macroeconomics, but that can't stop us from including a couple representative agent models anyway.

### `RepAgentConsumerType`: Basic representative agent model

This is a bog standard macroeconomic model in which a representative agent allocates their resources between consumption retained assets, which are used as capital in the subsequent period. Labor supply is assumed to be exogenous, and the "income" shocks are re-interpreted as labor productivity shocks.

The production technology is assumed to be Cobb-Douglas, and factor prices (wage rate and interest rate) are determined competitively as the marginal product of each factor.

### `RepAgentMarkovConsumerType`: Representative agent model with a discrete state

This model extends the prior one by adding a discrete state with exogenous dynamics given by some Markov matrix. There are fewer model objects that vary with the discrete state as compared to `MarkovConsumerType`, as there is no death in the model and the interest rate is endogenous.

The distribution of labor productivity shocks and the expected growth rate of labor productivity are the only things that can vary by the discrete state. The model can easily be extended to allow the capital depreciation factor and capital's share of production to also vary.

## `ConsAggShockModel`

This is the main module for *proper* heterogeneous agents macroeconomic models in HARK, and a good follow-up to the `RepAgentModel`. These models involve both idiosyncratic *and* aggregate productivity shocks, thus the name of the module.

### AggShockConsumerType: Standard model with aggregate and idiosyncratic income shocks

The `AggShockConsumerType` extends the `IndShockConsumerType` by adding a second state variable, *aggregate* market resources. Moreover, it adds *two* more shock variables: permanent and transitory *aggregate* shocks.

The economy produces output as a Cobb-Douglas aggregation of (inelastically supplied) labor and capital. Factor prices (the interest rate and wage rate) are determined endogenously in the model using the competitive outcome (marginal product of each factor).

When they make their consumption-saving decision, agents observe their own (normalized) market resources *and* (normalized) *aggregate* market resources. They have some expectation about how much will be saved in aggregate, and thus how much capital the economy will have at the start of the next period.

The microeconomic model for an `AggShockConsumerType` can be solved for arbitrary expectations about how aggregate market resources map to aggregate assets. To solve the model with *rational expectations*, one or more `AggShockConsumerType` instances must be combined with a `CobbDouglasEconomy`, a subclass of `Market`. The *economy itself* must have its `solve()` method run in order to iteratively find the aggregate saving rule that is consistent with rational expectations.

Alternatively, an `AggShockConsumerType` can be solved with a *trivial* grid of aggregate market resources if it is combined with a `SmallOpenEconomy`. In this macroeconomic framework, factor prices are determined *exogenously* (e.g. based on international conditions), but aggregate shocks still occur.

### `AggShockMarkovConsumerType`:

## `ConsBequestModel`

## `ConsRiskyAssetModel`

## `ConsPortfolioModel`

## `ConsWealthPortfolioModel`

## `ConsSequentialPortfolioModel`

## `ConsLaborModel`

## `ConsHealthModel`

## `ConsMedModel`

## `ConsPrefShockModel`